In [5]:
#!conda install -c conda-forge folium=0.5.0 --yes # comment/uncomment if not yet installed.
#!conda install -c conda-forge geopy --yes        # comment/uncomment if not yet installed

!pip install geocoder
!pip install folium
import numpy as np

import pandas as pd # library for data analsysis
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values # to get coordinates

import requests
import geocoder# library to handle requests
from bs4 import BeautifulSoup # library to parse HTML and XML documents

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print("Libraries  are imported.")

     |████████████████████████████████| 102kB 6.5MB/s ta 0:00:011
Libraries  are imported.


In [6]:
# send the GET request
data = requests.get("https://commons.wikimedia.org/wiki/Category:Suburbs_of_Bangalore").text

In [7]:
# parse data from the html into a beautifulsoup object
soup = BeautifulSoup(data, 'html.parser')

In [8]:
# create a list to store neighborhood data
neighborhoodList = []

In [9]:
# append the data into the list
for row in soup.find_all("div", class_="mw-category")[0].findAll("li"):
    neighborhoodList.append(row.text)

In [10]:
# create a new DataFrame from the list
kl_df = pd.DataFrame({"Neighborhood": neighborhoodList})
kl_df.head()

,Neighborhood
0,"► Agara, Bangalore‎ (2 C, 6 F)"
1,► Arekere‎ (5 F)
2,"► Banashankari‎ (1 C, 5 F)"
3,► Banaswadi‎ (2 F)
4,"► Basavanagudi‎ (5 C, 11 F)"


In [11]:
kl_df.shape

(59, 1)

In [12]:
# define a function to get coordinates
def get_latlng(neighborhood):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Bangalore, India'.format(neighborhood))
        lat_lng_coords = g.latlng
    return lat_lng_coords

In [13]:
# call the function to get the coordinates, store in a new list using list comprehension
coords = [ get_latlng(neighborhood) for neighborhood in kl_df["Neighborhood"].tolist() ]

In [14]:
coords

[[12.841272755642173, 77.48159794529579],
 [12.997990005183027, 77.61046999022852],
 [12.939339984645756, 77.54449996982913],
 [12.997840011025701, 77.61036999412363],
 [12.938980000000072, 77.57137000000006],
 [12.882450000000063, 77.62475000000006],
 [12.927350000000047, 77.67185000000006],
 [12.981180014545643, 77.62507000827469],
 [12.966180000000065, 77.58690000000007],
 [12.957999987042513, 77.60094001777514],
 [12.817530000000033, 77.67879000000005],
 [12.966223851801814, 77.60679161806351],
 [12.85545000008095, 77.73125002986342],
 [12.987949993921418, 77.6065900072486],
 [12.99787001823907, 77.61040000133698],
 [12.943290000000047, 77.65602000000007],
 [12.839890330733185, 77.67220534445676],
 [12.998940000000061, 77.61276000000004],
 [12.942790000000059, 77.54122000000007],
 [13.047449981350198, 77.63327999861599],
 [13.049810000000036, 77.58903000000004],
 [12.957450008465656, 77.60091001056179],
 [12.912160000000029, 77.64490000000006],
 [12.978220000000022, 77.633970000000

In [15]:
# create temporary dataframe to populate the coordinates into Latitude and Longitude
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])

In [16]:
# merge the coordinates into the original dataframe
kl_df['Latitude'] = df_coords['Latitude']
kl_df['Longitude'] = df_coords['Longitude']

In [17]:
# check the neighborhoods and the coordinates
print(kl_df.shape)
kl_df

(59, 3)


,Neighborhood,Latitude,Longitude
0,"► Agara, Bangalore‎ (2 C, 6 F)",12.841273,77.481598
1,► Arekere‎ (5 F),12.997990,77.610470
2,"► Banashankari‎ (1 C, 5 F)",12.939340,77.544500
3,► Banaswadi‎ (2 F),12.997840,77.610370
4,"► Basavanagudi‎ (5 C, 11 F)",12.938980,77.571370
5,"► Begur, Bangalore‎ (1 C, 6 F)",12.882450,77.624750
6,"► Bellandur‎ (1 C, 4 F)",12.927350,77.671850
7,► BEML‎ (7 F),12.981180,77.625070
8,"► Bengaluru Pete‎ (9 C, 4 F)",12.966180,77.586900
9,"► Bidadi‎ (2 C, 2 F)",12.958000,77.600940


In [18]:
# save the DataFrame as CSV file
kl_df.to_csv("kl_df.csv", index=False)

In [19]:
address = 'Bangalore, India'

geolocator = Nominatim(user_agent="http")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Bangalore, India {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Bangalore, India 12.9791198, 77.5912997.


In [20]:
# create map of Bangalore using latitude and longitude values
map_kl = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, neighborhood in zip(kl_df['Latitude'], kl_df['Longitude'], kl_df['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_kl)  
    
map_kl

In [21]:
# save the map as HTML file
map_kl.save('map_kl.html')

In [22]:
# define Foursquare Credentials and Version

CLIENT_ID = 'F5O1VFCM2XQPFMKSDI4MX4YKGMD5ODEEXEEFFHORIRXM0KZA' # your Foursquare ID
CLIENT_SECRET = 'AKOR0HDY0OQH00YBCK2HTO40CSNNCAMMHDKJI40PIBBPWFHS' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: F5O1VFCM2XQPFMKSDI4MX4YKGMD5ODEEXEEFFHORIRXM0KZA
CLIENT_SECRET:AKOR0HDY0OQH00YBCK2HTO40CSNNCAMMHDKJI40PIBBPWFHS


In [23]:
radius = 2000
LIMIT = 100

venues = []

for lat, long, neighborhood in zip(kl_df['Latitude'], kl_df['Longitude'], kl_df['Neighborhood']):
    
    # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    # return only relevant information for each nearby venue
    for venue in results:
        venues.append((
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [24]:
# convert the venues list into a new DataFrame
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['Neighborhood', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()


(3978, 7)


,Neighborhood,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,► Arekere‎ (5 F),12.99799,77.61047,Naturals Icecream,12.996912,77.611268,Ice Cream Shop
1,► Arekere‎ (5 F),12.99799,77.61047,Lassi Shop Downtown,13.000869,77.613731,Juice Bar
2,► Arekere‎ (5 F),12.99799,77.61047,Mangalore Pearl - Seafood Restaurant,12.994472,77.615551,Seafood Restaurant
3,► Arekere‎ (5 F),12.99799,77.61047,Richies - Frazer Town,13.000930,77.613150,Indian Restaurant
4,► Arekere‎ (5 F),12.99799,77.61047,Albert Bakery,12.996995,77.614308,Bakery


In [25]:
venues_df.groupby(["Neighborhood"]).count()

,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
Neighborhood,,,,,,
► Arekere‎ (5 F),100,100,100,100,100,100
► BEML‎ (7 F),100,100,100,100,100,100
"► Banashankari‎ (1 C, 5 F)",23,23,23,23,23,23
► Banaswadi‎ (2 F),100,100,100,100,100,100
"► Basavanagudi‎ (5 C, 11 F)",100,100,100,100,100,100
"► Begur, Bangalore‎ (1 C, 6 F)",16,16,16,16,16,16
"► Bellandur‎ (1 C, 4 F)",85,85,85,85,85,85
"► Bengaluru Pete‎ (9 C, 4 F)",100,100,100,100,100,100
"► Bidadi‎ (2 C, 2 F)",100,100,100,100,100,100


In [26]:
print('There are {} uniques categories.'.format(len(venues_df['VenueCategory'].unique())))

There are 198 uniques categories.


In [27]:
# print out the list of categories
venues_df['VenueCategory'].unique()[:50]

array(['Ice Cream Shop', 'Juice Bar', 'Seafood Restaurant',
       'Indian Restaurant', 'Bakery', 'BBQ Joint', 'Park',
       'Department Store', 'Tea Room', 'Fast Food Restaurant',
       'Pakistani Restaurant', 'Hyderabadi Restaurant', 'Pub',
       'Middle Eastern Restaurant', 'Pizza Place', 'Snack Place',
       'Shopping Mall', 'Café', 'Eastern European Restaurant',
       'Steakhouse', 'Sandwich Place', 'Concert Hall', 'Donut Shop',
       "Women's Store", 'Coffee Shop', 'Hotel', 'Chinese Restaurant',
       'Wine Shop', 'Market', 'Asian Restaurant', 'Badminton Court',
       'South Indian Restaurant', 'Clothing Store', 'Jewelry Store',
       'Music Venue', 'Bar', 'Mediterranean Restaurant', 'Event Space',
       'Trail', 'Pool Hall', 'Candy Store', 'Salad Place',
       'Breakfast Spot', 'Vegetarian / Vegan Restaurant',
       'Electronics Store', 'Travel & Transport', 'Metro Station',
       'Train Station', 'Toll Booth', 'Monument / Landmark'], dtype=object)

In [28]:
# check if the results contain "Shopping Mall"
"Neighborhood" in venues_df['VenueCategory'].unique()

True

In [29]:
# one hot encoding
kl_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
kl_onehot['Neighborhoods'] = venues_df['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [kl_onehot.columns[-1]] + list(kl_onehot.columns[:-1])
kl_onehot = kl_onehot[fixed_columns]

print(kl_onehot.shape)
kl_onehot.head()

(3978, 199)


,Neighborhoods,Accessories Store,Afghan Restaurant,Airport,American Restaurant,Andhra Restaurant,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Badminton Court,Bagel Shop,Bakery,Bar,Basketball Court,Bed & Breakfast,Beer Garden,Bengali Restaurant,Bike Shop,Bistro,Boarding House,Bookstore,Botanical Garden,Boutique,Bowling Alley,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Buffet,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Butcher,Cafeteria,Café,Candy Store,Capitol Building,Caribbean Restaurant,Chettinad Restaurant,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,Concert Hall,Cosmetics Shop,Coworking Space,Creperie,Cricket Ground,Cupcake Shop,Deli / Bodega,Department Store,Dessert Shop,Dhaba,Dim Sum Restaurant,Diner,Donut Shop,Eastern European Restaurant,Electronics Store,Event Space,Farmers Market,Fast Food Restaurant,Flea Market,Food,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gastropub,General Entertainment,German Restaurant,Golf Course,Grocery Store,Gym,Gym / Fitness Center,Health Food Store,Historic Site,Hockey Arena,Hookah Bar,Hospital,Hotel,Hotel Bar,Hyderabadi Restaurant,Ice Cream Shop,Indian Restaurant,Indian Sweet Shop,Indie Movie Theater,Irish Pub,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Karnataka Restaurant,Kebab Restaurant,Kerala Restaurant,Kids Store,Korean Restaurant,Lake,Light Rail Station,Liquor Store,Lounge,Maharashtrian Restaurant,Market,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Motorcycle Shop,Movie Theater,Moving Target,Multicuisine Indian Restaurant,Multiplex,Music Store,Music Venue,Neighborhood,Nightclub,Noodle House,North Indian Restaurant,Office,Outlet Store,Pakistani Restaurant,Paper / Office Supplies Store,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Platform,Playground,Plaza,Pool,Pool Hall,Pub,Punjabi Restaurant,Racetrack,Rajasthani Restaurant,Recreation Center,Resort,Restaurant,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Shoe Store,Shop & Service,Shopping Mall,Smoke Shop,Snack Place,Soccer Field,Soccer Stadium,South Indian Restaurant,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Steakhouse,Supermarket,Sushi Restaurant,Szechuan Restaurant,Tea Room,Tex-Mex Restaurant,Thai Restaurant,Theater,Toll Booth,Toy / Game Store,Track Stadium,Trail,Train Station,Travel & Transport,Turkish Restaurant,Udupi Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,► Arekere‎ (5 F),0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,► Arekere‎ (5 F),0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,► Arekere‎ (5 F),0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,

In [30]:
kl_grouped = kl_onehot.groupby(["Neighborhoods"]).mean().reset_index()

print(kl_grouped.shape)
kl_grouped

(56, 199)


,Neighborhoods,Accessories Store,Afghan Restaurant,Airport,American Restaurant,Andhra Restaurant,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Badminton Court,Bagel Shop,Bakery,Bar,Basketball Court,Bed & Breakfast,Beer Garden,Bengali Restaurant,Bike Shop,Bistro,Boarding House,Bookstore,Botanical Garden,Boutique,Bowling Alley,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Buffet,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Butcher,Cafeteria,Café,Candy Store,Capitol Building,Caribbean Restaurant,Chettinad Restaurant,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,Concert Hall,Cosmetics Shop,Coworking Space,Creperie,Cricket Ground,Cupcake Shop,Deli / Bodega,Department Store,Dessert Shop,Dhaba,Dim Sum Restaurant,Diner,Donut Shop,Eastern European Restaurant,Electronics Store,Event Space,Farmers Market,Fast Food Restaurant,Flea Market,Food,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gastropub,General Entertainment,German Restaurant,Golf Course,Grocery Store,Gym,Gym / Fitness Center,Health Food Store,Historic Site,Hockey Arena,Hookah Bar,Hospital,Hotel,Hotel Bar,Hyderabadi Restaurant,Ice Cream Shop,Indian Restaurant,Indian Sweet Shop,Indie Movie Theater,Irish Pub,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Karnataka Restaurant,Kebab Restaurant,Kerala Restaurant,Kids Store,Korean Restaurant,Lake,Light Rail Station,Liquor Store,Lounge,Maharashtrian Restaurant,Market,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Motorcycle Shop,Movie Theater,Moving Target,Multicuisine Indian Restaurant,Multiplex,Music Store,Music Venue,Neighborhood,Nightclub,Noodle House,North Indian Restaurant,Office,Outlet Store,Pakistani Restaurant,Paper / Office Supplies Store,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Platform,Playground,Plaza,Pool,Pool Hall,Pub,Punjabi Restaurant,Racetrack,Rajasthani Restaurant,Recreation Center,Resort,Restaurant,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Shoe Store,Shop & Service,Shopping Mall,Smoke Shop,Snack Place,Soccer Field,Soccer Stadium,South Indian Restaurant,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Steakhouse,Supermarket,Sushi Restaurant,Szechuan Restaurant,Tea Room,Tex-Mex Restaurant,Thai Restaurant,Theater,Toll Booth,Toy / Game Store,Track Stadium,Trail,Train Station,Travel & Transport,Turkish Restaurant,Udupi Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,► Arekere‎ (5 F),0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.010000,0.000000,0.000000,0.000000,0.020000,0.010000,0.00,0.030000,0.010000,0.00000,0.000000,0.00,0.00,0.000000,0.00,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.100000,0.010000,0.00,0.00,0.000000,0.030000,0.00,0.030000,0.000000,0.020000,0.010000,0.000000,0.00000,0.000000,0.00,0.00,0.000000,0.010000,0.000000,0.000000,0.000000,0.000000,0.010000,0.010000,0.000000,0.010000,0.000000,0.030000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.00,0.00,0.00,0.000000,0.030000,0.000000,0.010000,0.030000,0.240000,0.000000,0.000000,0.000000,0.000000,0.00,0.010000,0.020000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.010000,0.010000,0.000000,0.000000,0.000000,0.040000,0.000000,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.010000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.010000,0.000000,0.020000,0.000000,

In [31]:
len(kl_grouped[kl_grouped["Chinese Restaurant"] > 0])

42

In [32]:
kl_mall = kl_grouped[["Neighborhoods","Chinese Restaurant"]]

In [33]:
kl_mall.head()

,Neighborhoods,Chinese Restaurant
0,► Arekere‎ (5 F),0.03
1,► BEML‎ (7 F),0.05
2,"► Banashankari‎ (1 C, 5 F)",0.00
3,► Banaswadi‎ (2 F),0.03
4,"► Basavanagudi‎ (5 C, 11 F)",0.01


In [34]:
# set number of clusters
kclusters = 4

kl_clustering = kl_mall.drop(["Neighborhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(kl_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([1, 2, 0, 1, 0, 3, 0, 1, 0, 0], dtype=int32)

In [35]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
kl_merged = kl_mall.copy()

# add clustering labels
kl_merged["Cluster Labels"] = kmeans.labels_

In [36]:
kl_merged.rename(columns={"Neighborhoods": "Neighborhood"}, inplace=True)
kl_merged.head()

,Neighborhood,Chinese Restaurant,Cluster Labels
0,► Arekere‎ (5 F),0.03,1
1,► BEML‎ (7 F),0.05,2
2,"► Banashankari‎ (1 C, 5 F)",0.00,0
3,► Banaswadi‎ (2 F),0.03,1
4,"► Basavanagudi‎ (5 C, 11 F)",0.01,0


In [37]:
# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
kl_merged = kl_merged.join(kl_df.set_index("Neighborhood"), on="Neighborhood")

print(kl_merged.shape)
kl_merged.head() # check the last columns!

(56, 5)


,Neighborhood,Chinese Restaurant,Cluster Labels,Latitude,Longitude
0,► Arekere‎ (5 F),0.03,1,12.99799,77.61047
1,► BEML‎ (7 F),0.05,2,12.98118,77.62507
2,"► Banashankari‎ (1 C, 5 F)",0.00,0,12.93934,77.54450
3,► Banaswadi‎ (2 F),0.03,1,12.99784,77.61037
4,"► Basavanagudi‎ (5 C, 11 F)",0.01,0,12.93898,77.57137


In [38]:
# sort the results by Cluster Labels
print(kl_merged.shape)
kl_merged.sort_values(["Cluster Labels"], inplace=True)
kl_merged

(56, 5)


,Neighborhood,Chinese Restaurant,Cluster Labels,Latitude,Longitude
55,"► Yeswanthpur‎ (1 C, 1 F)",0.000000,0,13.029540,77.540220
25,"► Jayanagar, Bangalore‎ (1 C, 7 F)",0.010000,0,12.961610,77.597725
24,"► Jakkur‎ (2 C, 1 F)",0.000000,0,13.075640,77.603940
31,"► Krishnarajapura‎ (3 C, 3 F)",0.000000,0,13.000390,77.683680
21,"► Indiranagar‎ (4 C, 8 F)",0.010000,0,12.978220,77.633970
35,► Mahadevapura‎ (2 C),0.010000,0,12.958000,77.600940
17,"► HBR Layout‎ (1 C, 1 F)",0.000000,0,13.047450,77.633280
16,"► Girinagar‎ (1 C, 11 F)",0.000000,0,12.942790,77.541220
54,"► Yeshwantpur‎ (1 C, 6 F)",0.000000,0,13.029540,77.540220
13,"► Domlur‎ (5 C, 23 F)",0.000000,0,12.943290,77.656020


In [39]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(kl_merged['Latitude'], kl_merged['Longitude'], kl_merged['Neighborhood'], kl_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters



In [40]:
# save the map as HTML file
map_clusters.save('map_clusters.html')

In [41]:

kl_merged.loc[kl_merged['Cluster Labels'] == 0]

,Neighborhood,Chinese Restaurant,Cluster Labels,Latitude,Longitude
55,"► Yeswanthpur‎ (1 C, 1 F)",0.00,0,13.029540,77.540220
25,"► Jayanagar, Bangalore‎ (1 C, 7 F)",0.01,0,12.961610,77.597725
24,"► Jakkur‎ (2 C, 1 F)",0.00,0,13.075640,77.603940
31,"► Krishnarajapura‎ (3 C, 3 F)",0.00,0,13.000390,77.683680
21,"► Indiranagar‎ (4 C, 8 F)",0.01,0,12.978220,77.633970
35,► Mahadevapura‎ (2 C),0.01,0,12.958000,77.600940
17,"► HBR Layout‎ (1 C, 1 F)",0.00,0,13.047450,77.633280
16,"► Girinagar‎ (1 C, 11 F)",0.00,0,12.942790,77.541220
54,"► Yeshwantpur‎ (1 C, 6 F)",0.00,0,13.029540,77.540220
13,"► Domlur‎ (5 C, 23 F)",0.00,0,12.943290,77.656020


In [42]:
kl_merged.loc[kl_merged['Cluster Labels'] == 1]

,Neighborhood,Chinese Restaurant,Cluster Labels,Latitude,Longitude
42,► Rajajinagar‎ (3 C),0.025000,1,12.984580,77.555760
30,"► Koramangala‎ (1 C, 12 F)",0.030000,1,12.920040,77.625460
48,► Tippasandra‎ (1 C),0.030000,1,12.957450,77.600910
36,► Majestic (Bangalore)‎ (1 C),0.030000,1,12.957450,77.600910
37,"► Malleswaram‎ (4 C, 2 F)",0.020000,1,12.995000,77.573460
46,"► Shivajinagar‎ (1 C, 18 F)",0.030000,1,12.987200,77.604010
40,"► Murugeshpalya‎ (4 C, 12 F)",0.023256,1,12.955650,77.653350
34,"► Magadi‎ (2 C, 10 F)",0.030000,1,12.988230,77.605830
41,► Nagarbhavi‎ (1 C),0.030000,1,12.957450,77.600910
0,► Arekere‎ (5 F),0.030000,1,12.997990,77.610470


In [43]:
kl_merged.loc[kl_merged['Cluster Labels'] == 2]

,Neighborhood,Chinese Restaurant,Cluster Labels,Latitude,Longitude
1,► BEML‎ (7 F),0.050000,2,12.981180,77.625070
52,"► Whitefield, Bangalore‎ (4 C, 1 P, 22 F)",0.040541,2,12.975979,77.710317
49,"► UB City‎ (2 C, 19 F)",0.040000,2,12.987980,77.606650
47,► Thubarahalli‎ (97 F),0.047619,2,12.953500,77.721140
38,"► Marathahalli‎ (8 C, 1 P, 30 F)",0.042857,2,12.954660,77.707520
33,"► Madiwala‎ (1 C, 6 F)",0.050000,2,12.956610,77.613550
11,"► Devanahalli‎ (5 C, 2 F)",0.040000,2,12.987950,77.606590
28,"► Kodihalli, Bangalore‎ (1 C, 4 F)",0.043478,2,13.059760,77.576730
45,► Sahakara Nagar‎ (1 C),0.041667,2,13.060950,77.573980
32,► Kundalahalli‎ (96 F),0.049383,2,12.967520,77.715000


In [44]:
kl_merged.loc[kl_merged['Cluster Labels'] == 3]

,Neighborhood,Chinese Restaurant,Cluster Labels,Latitude,Longitude
5,"► Begur, Bangalore‎ (1 C, 6 F)",0.062500,3,12.88245,77.62475
50,"► Ulsoor‎ (4 C, 3 F)",0.060000,3,12.98908,77.62795
23,"► J. P. Nagar‎ (2 C, 1 F)",0.070000,3,12.90831,77.59024
39,► Mathikere‎ (1 C),0.075758,3,13.03033,77.55968
